In [1]:
import os
import sys
import numpy as np
import pandas as pd
sys.path.insert(0, '/home/jgutman/mvesc/ETL')

In [ ]:
from mvesc_utility_functions import *

In [17]:
with postgres_pgconnection_generator() as connection:
    connection.autocommit = True
    with connection.cursor() as cursor:
        schemas = ["'feature_scores'", "'predictions'"]
        schemas_str = ", ".join(schemas)
        pull_tables_to_delete = """select table_schema, table_name from information_schema.tables 
        where table_schema in ({}) limit 10000;""".format(schemas_str)
        cursor.execute(pull_tables_to_delete)
        tables_to_delete = cursor.fetchall()
        for schema, table in tables_to_delete:
            delete_table = """drop table {}."{}";""".format(schema, table)
            cursor.execute(delete_table)
        for schema in schemas:
            delete_schema = "drop schema if exists {};".format(schema.strip("'"))
            cursor.execute(delete_schema)